In [ ]:
from flask import Flask, request, jsonify
from pyngrok import ngrok
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
import io

# Подключаем Google Диск
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Модель ResNet18
model = models.resnet18(pretrained=False)
model.fc = nn.Linear(512, 23)
model.load_state_dict(torch.load("/content/drive/MyDrive/dermnet_model.pth", map_location=torch.device("cpu")))
model.eval()

# Классы
labels = [
    "Акне и розацеа",
    "Актинический кератоз Базалиома и другие злокачественные новообразования",
    "Атопический дерматит",
    "Буллезная болезнь",
    "Целлюлит, импетиго и другие бактериальные инфекции",
    "Экзема",
    "Экзантемы и лекарственные высыпания",
    "Выпадение волос, алопеция и другие заболевания волосяного покрова",
    "Герпес ВПЧ и другие ЗППП",
    "Легкие заболевания и нарушения пигментации",
    "Волчанка и другие заболевания соединительной ткани",
    "Меланома, рак кожи, невусы и родинки",
    "Грибок ногтей и другие заболевания ногтей",
    "Ядовитый плющ и другие контактные дерматиты",
    "Псориаз, красный плоский лишай и связанные с ними заболевания",
    "Чесотка, болезнь Лайма и другие инвазии и укусы",
    "Себорейные кератозы и другие доброкачественные опухоли",
    "Системное заболевание",
    "Опоясывающий лишай Кандидоз и другие грибковые инфекции",
    "Крапивница",
    "Сосудистые опухоли",
    "Васкулит",
    "Бородавки, моллюски и другие вирусные инфекции"
]


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])
# Завершаем предыдущие туннели, если есть
ngrok.kill()

# Инициализация Flask
app = Flask(__name__)

# Подключаем ngrok
ngrok.set_auth_token("NGROK_AUTH_TOKEN")
public_url = ngrok.connect(5000)
print("Сервер доступен по адресу:", public_url)


@app.route("/predict", methods=["POST"])
def predict():
    print("Запрос получен")

    if 'image' not in request.files:
        return jsonify({"error": "Нет изображения"}), 400

    file = request.files["image"]
    img_bytes = file.read()
    image = Image.open(io.BytesIO(img_bytes)).convert("RGB")
    input_tensor = transform(image).unsqueeze(0)

    with torch.no_grad():
        outputs = model(input_tensor)
        _, predicted = torch.max(outputs, 1)
        result = labels[predicted.item()]

    return jsonify({"result": result})
    app.run()

Сервер доступен по адресу: NgrokTunnel: "https://2613-35-184-73-157.ngrok-free.app" -> "http://localhost:5000"
